## Pretrained ResNet18 ImageNet Test

Pytorch Hub 공식 ResNet18 아키텍쳐를 이용

In [ ]:
# python library 
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models

In [ ]:
# GPU 장치사용 설정
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

#### ImageNet에 정의된 클래스 정보 가지고 오기

In [ ]:
from urllib.request import urlretrieve, Request, urlopen
import json

# 이미지넷(ImageNet) 에 정의된 1000개의 클래스 정보 가지고 오기
# imagenet_json, _ = urlretrieve('https://www.anishathalye.com/media/2017/07/25/imagenet.json' )
# 위의 코드 403 ERROR 발생
req = Request('http://www.anishathalye.com/media/2017/07/25/imagenet.json', headers={'User-Agent': 'Mozilla/5.0'})
imagenet_json= urlopen(req).read()

imagenet_labels = json.loads(imagenet_json)
imagenet_labels[18]

#### 이미지 처리 함수 정의 및 이미지 가져와 출력해보기
ResNet은 일반적으로 이미지에 대하여 **Resize, CenterCrop, ToTensor()**와 **입력 데이터 정규화**를 사용하는 모델 

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256), # 이미지 크기 변경
    transforms.CenterCrop(224), # 이미지의 중앙 부분을 잘라서 크기 조절
    transforms.ToTensor(), # torch.Tensor 형식으로 변경 [0, 255] -> [0, 1]
])

In [ ]:
import matplotlib.pyplot as plt
import PIL

In [ ]:
# 특정한 경로에서 이미지를 가져와 torch.Tensor로 변환하는 함수

def image_loader(path):
    image = PIL.Image.open(path)
    # 전처리 이후 네트워크 입력에 들어갈 이미지에 배치 목적의 차원 추가
    image = preprocess(image).unsqueeze(0)
    return image.to(device, torch.float) # GPU에 올리기 

In [ ]:
# 실제로 특정 URL에서 이미지를 불러오기 (얼룩 고양이)
# url = "http://www.image-net.org/nodes/10/02123045/6c/6c34fe7c9d846c33a2f1a9b47a766b44ab4ec70d.thumb"
# image_path, _ = urlretrieve(url)
# image = image_loader(image_path)
# 위의 경로 없어짐

image_path = "./data/cat_.jpeg"
image = image_loader(image_path)


In [ ]:
def imshow(tensor):
    # matplotlib는 CPU 기반이므로 CPU로 옮기기
    image = tensor.cpu().clone()
    # torch.Tensor에서 사용되는 배치 목적의 차원(dimension 제거)
    image = image.squeeze(0)
    # PIL 객체로 변경
    image = transforms.ToPILImage()(image)
    # 이미지를 화면에 출력, matplotlib는 [0, 1] 사이의 값이라도 정상적으로 이미지 출력 처리
    plt.imshow(image)

In [ ]:
plt.figure()
imshow(image)

#### Pretrained 된 모델 사용

In [ ]:
# 입력 데이터 정규화를 위한 클래스 정의

class Normalize(nn.Module) :
    def __init__(self, mean, std) :
        super(Normalize, self).__init__()
        self.register_buffer('mean', torch.Tensor(mean))
        self.register_buffer('std', torch.Tensor(std))
        
    def forward(self, input):
        mean = self.mean.reshape(1, 3, 1, 1)
        std = self.std.reshape(1, 3, 1, 1)
        return (input - mean) / std

**nn.Module.register_buffer**

<br>

일반적으로 모델 매개별수로 간주되지 않는 버퍼를 등록할 때사용
ex> BatchNorm에서 "running_mean"은 매개 변수는 아니지만 상태로서 사용 가능
 
```
Args:
    name (string): name of the buffer. The buffer can be accessed
        from this module using the given name
    tensor (Tensor): buffer to be registered.
```


In [ ]:
model = nn.Sequential( # 여러개의 layer를 순서대로 실행하기 위해 사용
    # 기본적인 ResNet18과 동일한 동작을 위하여 정규화 레이어 추가
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
    torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', weights=models.ResNet18_Weights.DEFAULT)
    ).to(device).eval() # 모델을 GPU로 옮기고 평가 모드로 변환

In [ ]:
# 기본적인 이미지를 실제 모델에 넣어 결과 확인
outputs = model(image)

# 확률을 계산하기 위해 softmax 함수
percentages = torch.nn.functional.softmax(outputs, dim=1)[0] * 100

# 가장 높은 값을 가지는 5개의 인덱스를 하나씩 확인 
for i in outputs[0].topk(5)[1]:
    print(f"인덱스 : {i.item()} / 클래스명 : {imagenet_labels[i]} / 확률 : {round(percentages[i].item(), 4)}%")

## CIFAR10 데이터 학습
- ResNet18 모델을 이용하여 학습

#### ResNet18 모델 정의 및 인스턴스 초기화

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os

# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        
        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) 
        
        # padding을 1 주기 대문에 높이와 너비가 동일
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # skip connection
        out = F.relu(out)
        
        return out
    
    
# ResNet Class 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        
        # 64개의 3x3 필터 (filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1) 
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks-1) # stride 값이 맨 처음 convolution 연산에만 적용될 수 있도록 ex > layer1 : [1, 1]
        # 즉, 첫 번째 convolution 연산에 의해서만 높이와 너비가 줄어들 수 있음 
        
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        
        return out
    

# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### Dataset 다운로드 및 불러오기

In [ ]:
import torchvision
import torchvision.transforms as transforms

transforms_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transforms_test = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transforms_train)
test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transforms_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=True, num_workers=4)

#### 환경 설정 및 학습 정의

In [ ]:
device = "cuda"

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net) 
# torch.nn.DataParallel 같은 모델을 지정한 여러 gpu device에 복사하고 입력 배치를 gpu 별로 쪼개 각 gpu 별로 forward/backward 연산 수행
cudnn.benchmark=True 
# cudnn.bebchmark가 True인 경우 cudnn의 benchmark를 통해 최적의 backend연산을 찾는 flag를 True로 하겠다는 의미

learning_rate = 0.1 # CIFAR10 dataset은 통상적으로 0.1부터 시작하여 줄여가는 방식 사용
file_name = "resnet18_cifar10.pt"

criterion=nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learnng_rate, momentum=0.9, weight_decay=0.0002)

def train(epoch):
    print("\n[ Train epoch : %d ]" % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        
        benign_outputs = net(inputs)
        loss = criterion(benign_outputs,targets)
        loss.backward()
        
        optimizer.step() # 전체 모델 parameter update
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)
        
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch : ', str(batch_idx))
            print("Current benign train accuracy : ", str(predicted.eq(targets).sum().item() / targets.size(0)))
            print("Current benign train loss : ", loss.item())
            
    print("\Total benign train accuracy : ", 100 * correct / total)
    print("Total benign train loss : ", train_loss / total)
    
    

def test(epoch):
    print("\n[ Test epoch : %d ]" % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)
        
        outputs = net(inputs)
        loss += criterion(outputs, targets).item()
        
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print("\Total benign test accuracy : ", 100 * correct / total)
    print("Total benign test loss : ", loss / total)
    
    state = {
        'net' : net.state_dict()
    }
    
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')
    
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    elif epoch >= 150:
        lr /= 10
    
    for param_group in optimizer.param_groups: # .param_groups...?
        param_group['lr'] = lr

#### Training

In [ ]:
for epoch in range(1, 201):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)

## 참고

In [ ]:
import torch, torchvision
import torchvision.models as models
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from PIL import Image

Deep Residual Learning for Image Recognition

Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun

https://arxiv.org/abs/1512.03385


Identity Mappings in Deep Residual Networks

Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun

https://arxiv.org/abs/1603.05027

In [ ]:
models.resnet18()

In [ ]:
models.resnet50()

In [ ]:
### Model
# resnet18 = models.resnet18(pretrained=True)
resnet18 = models.resnet18(weights="DEFAULT") # 자동으로 pretrain 된 모델 download

## Dataset
to_tensor = torchvision.transforms.Compose(
                [torchvision.transforms.ToTensor(),
               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])]
                                          )

cifar10 = torchvision.datasets.CIFAR10(root='./', download=True, transform=to_tensor)

dataloader = torch.utils.data.DataLoader(cifar10, batch_size=8, shuffle=True, num_workers=2)

In [ ]:
for idx, data in enumerate(dataloader):
    
    img, gt = data
    
    print(img.shape)
    
    scores = resnet18(img)
    
    print(scores.shape)
    break

## Fine-tuning

In [ ]:
## TODO: 1. replace the last FC layer for cifar10
### Hint: 1000 -> 10
model = resnet18(weig)


## TODO: 2. fine tuning the last classifier (FC layer) using the cifar 10 training set.



## TODO: 3. evaluation of the cifar 10 test set.



## Other models

In [ ]:
models.resnet50()

models.resnet101()

models.resnet152()


# Skip connection

https://pytorch.org/vision/0.8/_modules/torchvision/models/resnet.html


Basic Block vs. Bottleneck Block

![](./data/4_resnet_basicblock_bottleneck.png)

- Bottleneck 구조가 Image Classification에는 좋다고 함
- 다른 Task (Semantic Segmentation, Image 랩슨..?) 복자

# Batch normalization , Layer Normalization,  Instance Normalization, and Group Normalization

![](./data/group_norm.png) 



https://wandb.ai/wandb_fc/GroupNorm/reports/Group-Normalization-in-Pytorch-With-Examples---VmlldzoxMzU0MzMy



https://pytorch.org/docs/stable/generated/torch.nn.GroupNorm.html

- Batch norm : 한 channel에 대하여 normlize가 각각 됨
- Layer normalize는 모든 channel을 한번에 normalize 하지만 하나의 instance에 대해서만 normlize 함
    - Input Size가 정해져있을 때, RNN 같은 구조에서 효율적
- Instance norm은 Batch norm과 똑같지만 하나의 instance에 대해서 진행
    - Batch가 중요하지 않은 경우에 효과적, batch간 correlation이 없는 경우 
- Group norm은 channel 별로 group을 나누어서 진행

In [ ]:
import torch, torchvision
import torch.nn as nn
import torchvision.models as models
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from PIL import Image


Batch norm, layer norm, instance norm, group norm

https://pytorch.org/docs/stable/nn.html





https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html#torch.nn.LayerNorm

https://pytorch.org/docs/stable/generated/torch.nn.GroupNorm.html
    

In [ ]:
class BatchNorm(nn.Module):
    def __init__(self, in_channel, out_channels):
        super(BatchNorm, self).__init__()
        self.bn = nn.BatchNorm2d(in_channel)
        
    def forward(self,x):
        out = self.bn(x)  #[N, C, HW] -> [N, C, HW]
        
        
        return out

    
## For different sequences, e.g., RNN.
class LayerNorm(nn.Module):
    def __init__(self, in_shape, out_channels):
        super(LayerNorm, self).__init__()
        self.ln = nn.LayerNorm(in_shape, eps=1e-08)

    def forward(self,x):
        out = self.ln(x)  #[N, C, HW] -> [N, C, HW]

        
        return out

    
## For style transfer, domain adaptation.
class InstanceNorm(nn.Module):
    def __init__(self, in_channel, out_channels):
        super(InstanceNorm, self).__init__()
        self.In = nn.InstanceNorm2d(in_channel, eps=1e-08) 

    def forward(self,x):
        out = self.In(x)  #[N, C, HW] -> [N, C, HW]
        return out

    
## stable in small batch size.
class GroupNorm(nn.Module):
    def __init__(self, group_size, in_channel, out_channels):
        super(GroupNorm, self).__init__()
        self.gn = nn.GroupNorm(group_size, in_channel, eps=1e-08)  ## num_group and in_channel

    def forward(self,x):
        out = self.gn(x) #[N, C, HW] -> [N, C, HW]
        
        return out


In [ ]:
in_channel = 64
feature = torch.randn(8, in_channel, 120, 120)  ## temp tensor [B, C, H, W]


BN = BatchNorm(in_channel, out_channels=64)

out_feat = BN(feature)

print(out_feat.shape) # shape은 똑같지만 batch 전체가 nomalize 된 값으로 나옴
# 8개가 한번에 normalize 됨

In [ ]:
LN = LayerNorm(in_shape=list(feature.shape[1:]), out_channels=64)
# Batch 단위로 실행되는것이 아니기 때문에 shape이 들어가야 함

out_feat = LN(feature)

print(out_feat.shape)  # 8개 각각이 따로 normalize 됨

In [ ]:
feature.shape[1:]

In [ ]:
IN=InstanceNorm(in_channel, out_channels=64)

out_feat = IN(feature)

print(out_feat.shape) # 8, 64 제외한 120, 120이 한번에 normalizaion 됨

In [ ]:
GN=GroupNorm(group_size=2, in_channel=in_channel, out_channels=64)

out_feat = GN(feature)

print(out_feat.shape)  ## 32 / 32, 32개씩 나눠진 channel 들이 normalize 됨

GN=GroupNorm(group_size=4, in_channel=in_channel, out_channels=64)

out_feat = GN(feature)

print(out_feat.shape)  ## 16 / 16 / 16 / 16